# Identify and delimit species entries and their corresponding sections in the original text file

In [ ]:
import re

file_path = '/content/drive/MyDrive/PEDICULARIS.txt'

# Read file with ISO-8859-1 encoding
try:
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
    print(f"Successfully read file with {len(lines)} lines")
except Exception as e:
    print(f"Error reading file: {e}")

# Store unique species names
species_set = set()

# Pattern to match species entries (e.g., "1. Pedicularis name")
species_pattern = re.compile(r'^\d+\.\s+(Pedicularis\s+[a-z\-]+)')

# Extract species names
for line in lines:
    match = species_pattern.match(line.strip())
    if match:
        species_set.add(match.group(1))

# Count no of unique species
number_of_unique_species = len(species_set)
print(f"Number of unique Pedicularis species found: {number_of_unique_species}")

print("\nExample species found:")
for i, species in enumerate(sorted(list(species_set))[:10], 1):
    print(f"{i}. {species}")

# Identify and delimit sepcies descriptions in the original text file

In [ ]:
import re

def extract_species_descriptions(filepath, encoding='ISO-8859-1'):
    with open(filepath, 'r', encoding=encoding) as file:
        content = file.read()

    species_pattern = r'\d+\.\s+(Pedicularis\s+[^\n]+)\n\n(.+?)(?=\n\d+\.|$)'
    matches = re.findall(species_pattern, content, re.DOTALL)

    species_descriptions = {}
    for name, desc in matches:
        clean_name = name.split(",")[0].strip()
        species_descriptions[clean_name] = desc.strip().replace('\n', ' ')

    return species_descriptions

from google.colab import drive
drive.mount('/content/drive')

species_descriptions = extract_species_descriptions('/content/drive/MyDrive/PEDICULARIS.txt')

print(f"Extracted {len(species_descriptions)} species descriptions.")
print(list(species_descriptions.keys())[:5])

In [ ]:
import json

def save_species_descriptions(species_descriptions, filename="species_descriptions.json"):
    """Saves species descriptions to a JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(species_descriptions, f, indent=4, ensure_ascii=False)
        print(f"Species descriptions saved to {filename}")
    except Exception as e:
        print(f"Error saving species descriptions: {e}")

save_species_descriptions(species_descriptions)

# Prompts to extract standardized species traits (used to generate character matrix)

In [ ]:
import json
import time
import logging
from typing import Dict, List, Any, Optional
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

class BotanicalTraitExtractor:
    def __init__(
        self,
        api_key: str,
        model: str = "gpt-3.5-turbo",  # We can try more advanced models like gpt 4o or 4.5
        batch_size: int = 3,
        temperature: float = 0,
        max_retries: int = 3
    ):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.batch_size = batch_size
        self.temperature = temperature
        self.max_retries = max_retries
        self.token_usage = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}

        # Simplified trait schema with fewer categories for GPT-3.5
        self.trait_schema = {
            # Binary traits (0, 1, or "NA")
            "binary_traits": {
                "life_cycle": ["Life_cycle_annual", "Life_cycle_perennial", "Life_cycle_biennial"],
                "leaf_attachment": ["Leaf_clasping", "Leaf_not_clasping"],
                "stem_branching": ["Stem_branched", "Stem_unbranched", "Stem_cespitose"],
                "leaf_shape": ["Leaf_shape_linear", "Leaf_shape_lanceolate", "Leaf_shape_ovate",
                              "Leaf_shape_oblong", "Leaf_shape_elliptic"],
                "leaf_margin": ["Leaf_margin_entire", "Leaf_margin_dentate", "Leaf_margin_serrate",
                               "Leaf_margin_lobed", "Leaf_margin_pinnatifid"],
                "corolla_color": ["Corolla_yellow", "Corolla_purple", "Corolla_white",
                                 "Corolla_red", "Corolla_pink"],
                "filaments": ["Filaments_glabrous", "Filaments_pubescent", "Filaments_mixed"],
                "root_type": ["Root_fibrous", "Root_fascicled", "Root_fleshy"],
                "inflorescence_type": ["Inflorescence_racemose", "Inflorescence_capitate",
                                      "Inflorescence_spicate"],
                "capsule_shape": ["Capsule_shape_globose", "Capsule_shape_ovoid", "Capsule_shape_ellipsoid"]
            },

            # Numeric traits (either numeric value or null)
            "numeric_traits": {
                "stem_height": "Stem_height_cm",
                "ovary_size": "Ovary_size_mm",
                "calyx_lobes": "Calyx_lobes_count",
                "pedicel_length": "Pedicel_length_mm",
                "capsule_length": "Capsule_length_mm",
                "capsule_width": "Capsule_width_mm",
                "inflorescence_length": "Inflorescence_length_cm",
                "chromosome_number": "Chromosome_number_2n"
            },

            # List traits (arrays)
            "list_traits": {
                "flowering_months": "Fl_months",
                "fruiting_months": "Fr_months"
            }
        }

        # Flatten the schema for validation
        self.all_traits = []
        for trait_group in self.trait_schema["binary_traits"].values():
            self.all_traits.extend(trait_group)
        self.all_traits.extend(self.trait_schema["numeric_traits"].values())
        self.all_traits.extend(self.trait_schema["list_traits"].values())

    def _build_prompt(self, batch_desc: Dict[str, str]) -> str:
        """Build a concise prompt optimized for GPT-3.5-turbo's smaller context window"""
        # Simplified trait list for prompt
        trait_categories = []

        # Binary traits
        binary_traits_text = "BINARY TRAITS (0=absent, 1=present, NA=unknown):\n"
        for category, traits in self.trait_schema["binary_traits"].items():
            category_name = category.replace('_', ' ').title()
            trait_list = ", ".join([trait.split('_', 1)[1] for trait in traits])
            binary_traits_text += f"- {category_name}: {trait_list}\n"
        trait_categories.append(binary_traits_text)

        # Numeric traits
        numeric_traits_text = "NUMERIC TRAITS (use exact numbers or ranges as given):\n"
        for trait in self.trait_schema["numeric_traits"].values():
            numeric_traits_text += f"- {trait}\n"
        trait_categories.append(numeric_traits_text)

        # List traits
        list_traits_text = "LIST TRAITS:\n"
        list_traits_text += "- Fl_months (flowering months as 3-letter codes)\n"
        list_traits_text += "- Fr_months (fruiting months as 3-letter codes)"
        trait_categories.append(list_traits_text)

        # Add specific instructions about filament counts
        instructions = """
REQUIREMENTS:
1. Use ONLY the standard trait categories listed above
2. For binary traits: 1=present, 0=absent, "NA"=not mentioned
3. IMPORTANT: For filaments, if the description says "2 glabrous, 2 pubescent", do NOT use the number 2 - instead mark both Filaments_glabrous and Filaments_pubescent as 1 (present)
4. Keep numeric ranges exactly as given (e.g., "10-30")
5. Include all months mentioned for flowering/fruiting
6. IMPORTANT: You MUST include ALL trait categories for each species, even if not mentioned in description
7. For traits that have trait descriptions with multiple possible outcomes, e.g., "Corolla pale yellow to rose, often purple spotted", mark the presence of multiple colors as 1 (present), in the provided example, yellow, pink (given the similarity between pink and rose) and purple are marked as 1
8. For traits not mentioned in the species description, use "NA" for binary traits, null for numeric traits, and [] for lists
"""

        # Add filament-specific example in the example output

        example = """
{
  "Pedicularis sceptrum-carolinum": {
    "Life_cycle_perennial": 1,
    "Life_cycle_annual": 0,
    "Life_cycle_biennial": "NA",

    "Leaf_clasping": 0,                     // Leaf has a long petiole, not clasping
    "Leaf_not_clasping": 1,

    "Stem_height_cm": "60",
    "Stem_branched": 0,                     // "Stems often single" = unbranched
    "Stem_unbranched": 1,
    "Stem_cespitose": "NA",

    "Leaf_shape_linear": 1,                 // "linear-oblong"
    "Leaf_shape_lanceolate": 0,
    "Leaf_shape_ovate": 1,                  // Segments described as ovate
    "Leaf_shape_oblong": 1,                 // Segments and blade shape both oblong
    "Leaf_shape_elliptic": 0,

    "Leaf_margin_entire": 0,
    "Leaf_margin_dentate": 1,               // “double dentate”
    "Leaf_margin_serrate": 1,               // “lobes serrate” (on calyx but this might be ambiguous)
    "Leaf_margin_lobed": 1,                 // “pinnatilobate”
    "Leaf_margin_pinnatifid": 1,            // “pinnatifid to pinnatisect”

    "Corolla_yellow": 1,
    "Corolla_purple": 1,                    // “purple-red at apex of lower lip and galea”
    "Corolla_white": 0,
    "Corolla_red": 1,                       // “purple-red”
    "Corolla_pink": 0,                      // No mention of pink or “rose”

    "Ovary_size_mm": null,
    "Calyx_lobes_count": null,

    "Filaments_glabrous": 1,                // “Filaments glabrous apically”
    "Filaments_pubescent": 0,
    "Filaments_mixed": 0,

    "Root_fibrous": "NA",
    "Root_fascicled": "NA",
    "Root_fleshy": "NA",

    "Pedicel_length_mm": null,

    "Capsule_shape_globose": 1,             // Explicit
    "Capsule_shape_ovoid": 0,
    "Capsule_shape_ellipsoid": 0,

    "Capsule_length_mm": "20",              // Approximate conversion from “ca 2 cm”
    "Capsule_width_mm": null,

    "Inflorescence_racemose": 0,
    "Inflorescence_capitate": 0,
    "Inflorescence_spicate": 0,             // Not mentioned

    "Inflorescence_length_cm": "20",

    "Fl_months": ["Jun", "Jul", "Aug"],
    "Fr_months": ["Aug", "Sep"],

    "Chromosome_number_2n": "32"
  },

  "Pedicularis rudis": {
    "Life_cycle_perennial": 1,                // "Herbs perennial"
    "Life_cycle_annual": 0,
    "Life_cycle_biennial": "NA",

    "Leaf_clasping": 1,                       // "Stem leaves clasping"
    "Leaf_not_clasping": 0,

    "Stem_height_cm": ">100",                 // "to more than 1 m"
    "Stem_branched": 1,                       // "usually branched apically"
    "Stem_unbranched": 0,
    "Stem_cespitose": "NA",                   // No mention of tufted/cespitose habit

    "Leaf_shape_linear": 1,                   // "lanceolate-linear"
    "Leaf_shape_lanceolate": 1,               // Also from "oblong to lanceolate" segments
    "Leaf_shape_ovate": 0,                    // Not described
    "Leaf_shape_oblong": 1,                   // "oblong to lanceolate"
    "Leaf_shape_elliptic": 0,                 // Not described

    "Leaf_margin_entire": 0,
    "Leaf_margin_dentate": 1,                 // "margin double dentate"
    "Leaf_margin_serrate": 1,                 // "calyx lobes... serrate" (can be set to 0 if strict)
    "Leaf_margin_lobed": 0,                   // No mention of lobed/pinnatilobate etc.
    "Leaf_margin_pinnatifid": 1,              // "pinnatipartite" = a type of deeply pinnatifid

    "Corolla_yellow": 1,                      // "front yellow"
    "Corolla_purple": 1,                      // "galea purple-red apically"
    "Corolla_white": 1,                       // "Corolla white"
    "Corolla_red": 1,                         // part of "purple-red"
    "Corolla_pink": 0,                        // No mention of pink or rose tones

    "Ovary_size_mm": null,                    // Not described
    "Calyx_lobes_count": 5,                   // "lobes 5"

    "Filaments_glabrous": 1,                  // "Filaments glabrous"
    "Filaments_pubescent": 0,
    "Filaments_mixed": 0,

    "Root_fibrous": "NA",                     // Not described
    "Root_fascicled": "NA",
    "Root_fleshy": "NA",

    "Pedicel_length_mm": null,                // Not described

    "Capsule_shape_globose": 0,
    "Capsule_shape_ovoid": 1,                 // "Capsule broadly ovoid"
    "Capsule_shape_ellipsoid": 0,             // Not described

    "Capsule_length_mm": "13",                // "ca 1.3 cm"
    "Capsule_width_mm": "8",                  // "8 mm"

    "Inflorescence_racemose": 0,
    "Inflorescence_capitate": 0,
    "Inflorescence_spicate": 1,               // "Inflorescences long spicate"

    "Inflorescence_length_cm": ">30",         // "more than 30 cm"

    "Fl_months": ["Jul", "Aug"],              // "Fl. Jul–Aug"
    "Fr_months": ["Aug", "Sep"],              // "fr. Aug–Sep"

    "Chromosome_number_2n": null              // Not described
  },

  "Pedicularis verticillata": {
    "Life_cycle_perennial": 1,                // "Herbs perennial"
    "Life_cycle_annual": 0,
    "Life_cycle_biennial": "NA",

    "Leaf_clasping": "NA",                    // Not mentioned
    "Leaf_not_clasping": "NA",                // Not mentioned either

    "Stem_height_cm": "15-35",                // (5--)15–35 cm

    "Stem_branched": "NA",                    // Stems 1–7, but no mention of branching
    "Stem_unbranched": "NA",                  // Not directly stated
    "Stem_cespitose": "NA",                   // Not described as tufted

    "Leaf_shape_linear": 1,                   // "linear-lanceolate", "linear-oblong"
    "Leaf_shape_lanceolate": 1,
    "Leaf_shape_ovate": 1,                    // "triangular-ovate" (in segments)
    "Leaf_shape_oblong": 1,                   // "linear-oblong"
    "Leaf_shape_elliptic": 0,                 // Not mentioned

    "Leaf_margin_entire": 0,
    "Leaf_margin_dentate": 1,                 // "incised-dentate"
    "Leaf_margin_serrate": 1,                 // "Calyx lobes serrate"
    "Leaf_margin_lobed": 1,                   // Based on "pinnatifid" (deeply lobed)
    "Leaf_margin_pinnatifid": 1,              // "pinnatifid to pinnatisect"

    "Corolla_yellow": 0,
    "Corolla_purple": 1,                      // "Corolla purple"
    "Corolla_white": 0,
    "Corolla_red": 0,
    "Corolla_pink": 0,

    "Ovary_size_mm": null,                    // Not mentioned
    "Calyx_lobes_count": 5,                   // Range given is 3–5, use max as conservative
                                              // Optionally set to null if ambiguous

    "Filaments_glabrous": 1,                  // Only anterior pair pubescent → others glabrous
    "Filaments_pubescent": 1,                 // Anterior filament pair is pubescent
    "Filaments_mixed": 0,

    "Root_fibrous": "NA",
    "Root_fascicled": "NA",
    "Root_fleshy": "NA",

    "Pedicel_length_mm": null,                // Not described

    "Capsule_shape_globose": 0,
    "Capsule_shape_ovoid": 0,
    "Capsule_shape_ellipsoid": 0,             // None described
                                              // Capsule is "lanceolate", not one of the above shapes

    "Capsule_length_mm": "10-15",             // "1–1.5 cm"
    "Capsule_width_mm": "4-5",                // "4–5 mm"

    "Inflorescence_racemose": 1,              // "Inflorescences racemose"
    "Inflorescence_capitate": 0,              // Incorrect in original
    "Inflorescence_spicate": 0,               // Not described as a spike

    "Inflorescence_length_cm": null,          // No length given

    "Fl_months": ["Jun", "Jul", "Aug"],       // "Fl. Jun–Aug"
    "Fr_months": ["Jul", "Aug", "Sep"],       // "fr. Jul–Sep"

    "Chromosome_number_2n": "12 + 0–2f"       // Explicit
  },

  "Pedicularis roylei": {
    "Life_cycle_perennial": 1,                // "Herbs perennial"
    "Life_cycle_annual": 0,
    "Life_cycle_biennial": "NA",

    "Leaf_clasping": "NA",                    // Not explicitly stated
    "Leaf_not_clasping": "NA",                // Not stated either

    "Stem_height_cm": "7-15",                 // From "(4--)7–15 cm"

    "Stem_branched": "NA",                    // “1 to several stems” ≠ branching
    "Stem_unbranched": "NA",                  // Not clearly stated
    "Stem_cespitose": 1,                      // "Basal leaves cespitose and persistent"

    "Leaf_shape_linear": 0,
    "Leaf_shape_lanceolate": 1,               // “lanceolate-oblong”, “lanceolate to oblong” segments
    "Leaf_shape_ovate": 1,                    // “ovate-oblong” present
    "Leaf_shape_oblong": 1,                   // Appears in both leaf blade and segments
    "Leaf_shape_elliptic": 0,                 // Not described

    "Leaf_margin_entire": 0,
    "Leaf_margin_dentate": 1,                 // “incised-dentate”
    "Leaf_margin_serrate": 1,                 // “serrate” mentioned (on calyx lobes)
    "Leaf_margin_lobed": 1,                   // “pinnatifid” calyx lobes, plus deeply divided leaves
    "Leaf_margin_pinnatifid": 1,              // “pinnatipartite” in leaf blade

    "Corolla_yellow": 0,
    "Corolla_purple": 1,                      // “purple-red”
    "Corolla_white": 0,
    "Corolla_red": 1,                         // “purple-red”
    "Corolla_pink": 0,

    "Ovary_size_mm": null,                    // Not described
    "Calyx_lobes_count": 5,                   // “lobes 5”

    "Filaments_glabrous": 1,                  // “Filaments glabrous”
    "Filaments_pubescent": 0,
    "Filaments_mixed": 0,

    "Root_fibrous": 0,
    "Root_fascicled": 0,
    "Root_fleshy": 1,                          // “Roots fleshy, to 4–8 mm in diam”

    "Pedicel_length_mm": "2",                 // “Pedicel ca 2 mm”

    "Capsule_shape_globose": 0,
    "Capsule_shape_ovoid": 1,                 // “ovoid-lanceolate”
    "Capsule_shape_ellipsoid": 0,             // Not described

    "Capsule_length_mm": "12",                // “ca 1.2 cm”
    "Capsule_width_mm": null,                 // Not given directly

    "Inflorescence_racemose": 1,              // “Inflorescences racemose”
    "Inflorescence_capitate": 0,
    "Inflorescence_spicate": 0,

    "Inflorescence_length_cm": "6",           // “to 6 cm”

    "Fl_months": ["Jul", "Aug"],              // “Fl. Jul–Aug”
    "Fr_months": ["Aug", "Sep"],              // “fr. Aug–Sep”

    "Chromosome_number_2n": null              // Not mentioned
  }
}"""


        # Combine all components into a compact prompt
        return f"""You are a botanical taxonomy expert extracting standardized traits from plant descriptions.

{' '.join(trait_categories)}

{instructions}

COMPLETE EXAMPLE WITH ALL REQUIRED TRAITS:
{example}

CRITICAL INSTRUCTIONS:
1. You MUST include EVERY trait listed in the example for EACH species
2. For any trait not mentioned in the description, use "NA" for binary traits, null for numeric traits, and [] for lists
3. Do not omit any traits, even if they're not mentioned in the description
4. For filaments described as "2 glabrous, 2 pubescent", set Filaments_glabrous=1 and Filaments_pubescent=1 (not 2)
5. NEVER use numeric values other than 0 or 1 for binary traits - even if counts are mentioned in the description
6. Your JSON response must include complete data for ALL species

Species descriptions:
{json.dumps(batch_desc, indent=2)}"""

    def _attempt_json_repair(self, content: str) -> str:
        """Attempt to repair common JSON formatting issues from GPT-3.5 responses"""
        # Basic repair for unclosed quotes and brackets
        fixed_content = content

        # Remove /* Comments */ that might be in the JSON
        import re
        fixed_content = re.sub(r'/\*.*?\*/', '', fixed_content, flags=re.DOTALL)

        # Handle missing closing braces at the end
        open_braces = fixed_content.count('{')
        close_braces = fixed_content.count('}')
        if open_braces > close_braces:
            fixed_content += '}' * (open_braces - close_braces)

        # Handle unescaped quotes in strings
        in_string = False
        in_escape = False
        repaired = []

        for i, char in enumerate(fixed_content):
            if char == '\\' and not in_escape:
                in_escape = True
                repaired.append(char)
            elif in_escape:
                in_escape = False
                repaired.append(char)
            elif char == '"':
                in_string = not in_string
                repaired.append(char)
            else:
                repaired.append(char)

        # If we end while still in a string, add closing quote
        if in_string:
            repaired.append('"')

        fixed_content = ''.join(repaired)

        # Remove JS-style comments that GPT-3.5 sometimes includes
        lines = fixed_content.split('\n')
        cleaned_lines = [line for line in lines if not line.strip().startswith('//')]
        fixed_content = '\n'.join(cleaned_lines)

        # Try to fix truncated JSONs with missing closing brackets
        if not fixed_content.strip().endswith('}'):
            fixed_content = fixed_content.strip() + '}'

        return fixed_content

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_exception_type((json.JSONDecodeError, Exception))
    )
    def _call_api(self, prompt: str) -> Dict[str, Any]:
        """Call the OpenAI API with retry logic and special handling for GPT-3.5"""
        try:
            system_message = """You are a botanical taxonomy expert. Output only valid JSON.

IMPORTANT FORMATTING RULES:
1. Binary traits must ONLY have values of 0, 1, or "NA" - NEVER use numbers greater than 1
2. CRITICAL: For filaments, if a description mentions "2 glabrous filaments", assign value 1 to Filaments_glabrous, NOT 2
3. CRITICAL: For filaments, if a description mentions "2 pubescent filaments", assign value 1 to Filaments_pubescent, NOT 2
4. Your response must be valid JSON with no comments
5. Remember that for all binary traits, only use 0, 1, or "NA" as values
"""

            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ],
                response_format={"type": "json_object"},
                temperature=self.temperature
            )

            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                self.token_usage["prompt_tokens"] += response.usage.prompt_tokens
                self.token_usage["completion_tokens"] += response.usage.completion_tokens
                self.token_usage["total_tokens"] += response.usage.total_tokens

            content = response.choices[0].message.content

            # Try to parse the JSON response
            try:
                return json.loads(content)
            except json.JSONDecodeError as json_err:
                # Log detailed information about the JSON error
                error_context = content[max(0, json_err.pos - 50):min(len(content), json_err.pos + 50)]
                logger.warning(f"JSON decode error at position {json_err.pos}: {json_err.msg}")
                logger.warning(f"Error context: '...{error_context}...'")

                # Try to fix the JSON - GPT-3.5 is more likely to need this
                fixed_content = self._attempt_json_repair(content)
                if fixed_content != content:
                    try:
                        return json.loads(fixed_content)
                    except json.JSONDecodeError:
                        pass  # If still fails, continue to error handling

                logger.error(f"Failed to parse API response as JSON. Response truncated: {content[:500]}...")
                raise

        except Exception as e:
            logger.error(f"API call failed: {str(e)}")
            raise

    def _validate_traits(self, species_traits: Dict[str, Any], species_name: str) -> Dict[str, Any]:
        """Validate and standardize trait values"""
        validated_traits = {}

        # Pre-fill all traits with appropriate defaults first to avoid missing traits
        for trait_group in self.trait_schema["binary_traits"].values():
            for trait in trait_group:
                validated_traits[trait] = "NA"

        for trait in self.trait_schema["numeric_traits"].values():
            validated_traits[trait] = None

        for trait in self.trait_schema["list_traits"].values():
            validated_traits[trait] = []

        # Now check for traits in the response and update accordingly
        for trait in self.all_traits:
            if trait not in species_traits:
                # Trait already has default value, just log if needed
                if logger.isEnabledFor(logging.DEBUG):  # Only log at debug level to reduce noise
                    logger.debug(f"Missing trait '{trait}' for species '{species_name}'")
            else:
                value = species_traits[trait]

                # Validate and standardize binary traits
                if any(trait in trait_list for trait_list in self.trait_schema["binary_traits"].values()):
                    # Special handling for filament traits with counts (e.g., "2 glabrous, 2 pubescent")
                    if trait.startswith("Filaments_") and isinstance(value, (int, float)) and value > 1:
                        # If value is a count (like 2), treat as present (1)
                        validated_traits[trait] = 1
                        logger.info(f"Converted filament count {value} to binary presence (1) for trait '{trait}'")
                    elif value not in [0, 1, "NA"]:
                        if isinstance(value, str) and value.lower() in ["true", "yes", "present"]:
                            validated_traits[trait] = 1
                        elif isinstance(value, str) and value.lower() in ["false", "no", "absent"]:
                            validated_traits[trait] = 0
                        elif value is None:
                            validated_traits[trait] = "NA"
                        else:
                            # Try to convert string numbers to appropriate values
                            if isinstance(value, str) and value.isdigit():
                                num_value = int(value)
                                if num_value > 0:
                                    validated_traits[trait] = 1
                                    logger.info(f"Converted string number '{value}' to binary presence (1) for trait '{trait}'")
                                elif num_value == 0:
                                    validated_traits[trait] = 0
                                else:
                                    validated_traits[trait] = "NA"
                            else:
                                logger.warning(f"Invalid binary value '{value}' for trait '{trait}' - defaulting to 'NA'")
                                validated_traits[trait] = "NA"
                    else:
                        validated_traits[trait] = value

                # Validate and standardize numeric traits
                elif trait in self.trait_schema["numeric_traits"].values():
                    if value is None or value == "NA":
                        validated_traits[trait] = None
                    elif isinstance(value, (int, float)) or (isinstance(value, str) and "-" in value):
                        validated_traits[trait] = value
                    elif isinstance(value, str):
                        # Try to convert string to number if appropriate
                        try:
                            if value.replace(".", "", 1).isdigit():
                                if "." in value:
                                    validated_traits[trait] = float(value)
                                else:
                                    validated_traits[trait] = int(value)
                            else:
                                validated_traits[trait] = value
                        except:
                            validated_traits[trait] = value
                    else:
                        logger.warning(f"Unexpected value type for numeric trait '{trait}': {type(value)}")
                        validated_traits[trait] = value

                # Validate and standardize list traits
                elif trait in self.trait_schema["list_traits"].values():
                    if not isinstance(value, list):
                        if value is None or value == "NA":
                            validated_traits[trait] = []
                        elif isinstance(value, str):
                            if value.startswith("[") and value.endswith("]"):
                                try:
                                    validated_traits[trait] = json.loads(value)
                                except:
                                    validated_traits[trait] = []
                            else:
                                # Attempt to split comma-separated string
                                values = [v.strip(' "\'') for v in value.split(",")]
                                validated_traits[trait] = values
                        else:
                            logger.warning(f"Invalid list value '{value}' for trait '{trait}' - defaulting to []")
                            validated_traits[trait] = []
                    else:
                        validated_traits[trait] = value

                else:
                    # For any unexpected traits
                    validated_traits[trait] = value

        # Add any traits in the original that weren't in our validation set
        for trait, value in species_traits.items():
            if trait not in validated_traits and trait not in ["NA", None]:
                validated_traits[trait] = value

        return validated_traits

    def _process_in_chunks(self, species_data: List[tuple], chunk_size: int = 1) -> Dict[str, Dict[str, Any]]:
        """Process species in even smaller chunks when needed"""
        results = {}

        for i in range(0, len(species_data), chunk_size):
            chunk = species_data[i:i+chunk_size]
            chunk_dict = {species: desc for species, desc in chunk}

            logger.info(f"Processing chunk of {len(chunk)} species...")
            prompt = self._build_prompt(chunk_dict)

            try:
                traits_chunk = self._call_api(prompt)

                for species, _ in chunk:
                    if traits_chunk and species in traits_chunk:
                        validated_traits = self._validate_traits(traits_chunk[species], species)
                        results[species] = validated_traits
                        logger.info(f"Successfully processed '{species}'")
                    else:
                        logger.warning(f"Species '{species}' missing from API response in chunk")
                        # Create template with default values for missing species
                        default_traits = self._create_default_traits()
                        results[species] = default_traits

                time.sleep(1)  # Rate limiting

            except Exception as e:
                logger.error(f"Error processing chunk: {str(e)}")
                # Create default entries for all species in the failed chunk
                for species, _ in chunk:
                    if species not in results:
                        default_traits = self._create_default_traits()
                        results[species] = default_traits

        return results

    def _create_default_traits(self) -> Dict[str, Any]:
        """Create a default traits dictionary with appropriate empty values"""
        default_traits = {}

        # Set defaults for all trait types
        for binary_category in self.trait_schema["binary_traits"].values():
            for trait in binary_category:
                default_traits[trait] = "NA"

        for trait in self.trait_schema["numeric_traits"].values():
            default_traits[trait] = None

        for trait in self.trait_schema["list_traits"].values():
            default_traits[trait] = []

        return default_traits

    def extract_traits(self, descriptions: Dict[str, str]) -> Dict[str, Dict[str, Any]]:
        """Extract standardized traits from species descriptions with GPT-3.5 optimizations"""
        species_names = list(descriptions.keys())
        batches = [species_names[i:i+self.batch_size] for i in range(0, len(species_names), self.batch_size)]
        all_traits = {}
        failed_batches = []

        for idx, batch in enumerate(batches):
            logger.info(f"Processing batch {idx+1}/{len(batches)} with {len(batch)} species...")

            batch_desc = {name: descriptions[name] for name in batch}
            prompt = self._build_prompt(batch_desc)

            try:
                traits_batch = self._call_api(prompt)

                # Validate each species in the batch
                for species in batch:
                    if traits_batch and species in traits_batch:
                        # Validate traits for this species
                        validated_traits = self._validate_traits(traits_batch[species], species)
                        all_traits[species] = validated_traits
                    else:
                        logger.warning(f"Species '{species}' missing from API response")
                        failed_batches.append((species, descriptions[species]))

                # Rate limiting
                time.sleep(1)

            except Exception as e:
                logger.error(f"Error processing batch {idx+1}: {str(e)}")
                # Mark all species in this batch as failed
                for species in batch:
                    if species not in all_traits:
                        failed_batches.append((species, descriptions[species]))

        # Process failed species in even smaller chunks or individually
        if failed_batches:
            logger.info(f"Retrying {len(failed_batches)} failed species in smaller chunks...")
            chunk_results = self._process_in_chunks(failed_batches)
            all_traits.update(chunk_results)

        # Report token usage
        logger.info(f"Token usage summary: {self.token_usage}")
        return all_traits


def extract_traits_with_gpt(
    descriptions: Dict[str, str],
    api_key: str,
    batch_size: int = 3,  # Smaller default batch size for GPT-3.5
    model: str = "gpt-3.5-turbo",  # Default to GPT-3.5-turbo
    temperature: float = 0
) -> Dict[str, Dict[str, Any]]:
    """
    Extract standardized botanical traits from species descriptions using GPT-3.5-turbo.

    Parameters:
    - descriptions: Dictionary mapping species names to description text
    - api_key: OpenAI API key
    - batch_size: Number of species to process in each batch (default: 3)
    - model: OpenAI model to use (default: "gpt-3.5-turbo")
    - temperature: Sampling temperature (default: 0 for maximum determinism)

    Returns:
    - Dictionary mapping species names to their standardized traits
    """
    extractor = BotanicalTraitExtractor(
        api_key=api_key,
        model=model,
        batch_size=batch_size,
        temperature=temperature
    )
    return extractor.extract_traits(descriptions)

# Run model

In [ ]:
# Provide API key
api_key = ""
traits = extract_traits_with_gpt(species_descriptions, api_key, model = "gpt-3.5-turbo")

# Save traits

In [ ]:
import json
import os

# "traits" is already defined

def save_traits(traits):
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

  # file path
  file_path = "/content/drive/MyDrive/Complete_Pedicularis_traits.json"

  # Create directory if absent
  os.makedirs(os.path.dirname(file_path), exist_ok=True)

  # Save traits to a JSON file
  with open(file_path, 'w') as f:
    json.dump(traits, f, indent=4)

# Example
save_traits(traits)

# Load traits

In [ ]:
import json

def load_traits(file_path):
    """
    Load traits from a JSON file.

    Parameters:
    - file_path: Path to the JSON file containing the traits

    Returns:
    - Dictionary containing the loaded traits
    """
    with open(file_path, 'r') as f:
        traits = json.load(f)
    return traits

# Example
file_path = "/content/drive/MyDrive/Complete_Pedicularis_traits.json"
traits = load_traits(file_path)

# Print number of species loaded
print(f"Loaded traits for {len(traits)} species")

# Print a sample of the first species
if traits:
    first_species = next(iter(traits))
    print(f"Sample species: {first_species}")
    print(f"Number of traits: {len(traits[first_species])}")

# Generate dichotomous key

In [ ]:
import json
from typing import Dict, List, Any, Tuple, Optional, Set
import pandas as pd
import numpy as np
from IPython.display import display, HTML

class DichotomousKey:
    def __init__(self, traits_data: Dict[str, Dict[str, Any]]):
        """
        Initialize the dichotomous key with trait data

        Parameters:
        - traits_data: Dictionary mapping species names to trait dictionaries
        """
        self.traits_data = traits_data
        self.all_species = list(traits_data.keys())
        self.all_traits = self._get_all_traits()
        self.binary_traits = self._get_binary_traits()
        self.decision_tree = None

    def _get_all_traits(self) -> List[str]:
        """Get all trait names from the dataset."""
        all_traits = set()
        for species_traits in self.traits_data.values():
            all_traits.update(species_traits.keys())
        return list(all_traits)

    def _get_binary_traits(self) -> List[str]:
        """Get only binary traits (those with values 0, 1, or 'NA')."""
        binary_traits = []

        # Look at first species to identify binary traits
        first_species = next(iter(self.traits_data.values()))

        for trait, value in first_species.items():
            # Skip numeric traits and list traits
            if (trait.endswith('_cm') or trait.endswith('_mm') or
                trait.endswith('_count') or trait == 'Chromosome_number_2n' or
                trait.endswith('_months')):
                continue

            # Add to binary traits
            binary_traits.append(trait)

        return binary_traits

    def _find_best_trait(self, species_list: List[str]) -> Optional[str]:
        """
        Find the most informative trait for the current set of species.

        Parameters:
        - species_list: List of species names to consider

        Returns:
        - The most informative trait name or None if no good trait is found
        """
        if len(species_list) <= 1:
            return None  # No need to split further

        best_trait = None
        best_score = -float('inf')

        # Calculate how well each trait splits the remaining species
        for trait in self.binary_traits:
            # Count species with trait = 1 and trait = 0 and NA
            yes_count = sum(1 for s in species_list
                          if self.traits_data[s][trait] == 1)
            no_count = sum(1 for s in species_list
                         if self.traits_data[s][trait] == 0)
            na_count = sum(1 for s in species_list
                         if self.traits_data[s][trait] == "NA")

            # Skip traits where all species have the same value or mostly NA
            if yes_count == 0 or no_count == 0:
                continue

            # Skip traits with too many NA values
            if na_count > 0.5 * len(species_list):
                continue

            # Calculate information gain (using gini impurity)
            total = yes_count + no_count
            gini = 1 - ((yes_count/total)**2 + (no_count/total)**2)

            # For balance, we want traits that split as evenly as possible
            balance = -abs(0.5 - (yes_count / total))

            # Penalize traits with many NA values slightly
            na_penalty = -0.1 * (na_count / len(species_list))

            # Combine metrics (higher is better)
            score = balance + gini + na_penalty

            if score > best_score:
                best_score = score
                best_trait = trait

        return best_trait

    def _build_tree(self, species_list: List[str], depth: int = 0,
                   max_depth: int = 20) -> Dict[str, Any]:
        """
        Recursively build a decision tree for the species.

        Parameters:
        - species_list: List of species to classify
        - depth: Current depth in the tree
        - max_depth: Maximum depth to prevent infinite recursion

        Returns:
        - A nested dictionary representing the decision tree
        """
        # Base cases: leaf nodes
        if len(species_list) == 0:
            return {"type": "leaf", "species": []}

        if len(species_list) == 1 or depth >= max_depth:
            return {"type": "leaf", "species": species_list}

        # Find the best trait to split on
        trait = self._find_best_trait(species_list)

        # If no good trait found, return leaf node
        if trait is None:
            return {"type": "leaf", "species": species_list}

        # Split species based on trait
        yes_species = [s for s in species_list
                      if self.traits_data[s][trait] == 1]
        no_species = [s for s in species_list
                     if self.traits_data[s][trait] == 0]

        # Create internal node
        return {
            "type": "node",
            "trait": trait,
            "display_name": self._format_trait_name(trait),
            "yes": self._build_tree(yes_species, depth + 1, max_depth),
            "no": self._build_tree(no_species, depth + 1, max_depth)
        }

    def _format_trait_name(self, trait: str) -> str:
        """Format trait name for better display."""
        return trait.replace('_', ' ').title()

    def build(self) -> None:
        """Build the dichotomous key."""
        self.decision_tree = self._build_tree(self.all_species)
        print(f"Dichotomous key built for {len(self.all_species)} species with {len(self.binary_traits)} binary traits.")

    def _follow_path(self, node: Dict[str, Any], decisions: Dict[str, bool]) -> List[str]:
        """
        Follow a path in the tree based on decisions and return resulting species.
        Also handles skipped decisions by exploring both paths.

        Parameters:
        - node: Current tree node
        - decisions: Dictionary mapping traits to True/False decisions

        Returns:
        - List of species resulting from following this path
        """
        # If we've reached a leaf, return its species
        if node["type"] == "leaf":
            return node["species"]

        trait = node["trait"]

        # Check if we have a decision for this trait
        if trait in decisions:
            # Follow the path based on our decision
            if decisions[trait]:
                return self._follow_path(node["yes"], decisions)
            else:
                return self._follow_path(node["no"], decisions)
        else:
            # If trait was skipped, follow both paths and combine results
            yes_results = self._follow_path(node["yes"], decisions)
            no_results = self._follow_path(node["no"], decisions)
            return list(set(yes_results + no_results))

    def identify(self, interactive: bool = True) -> None:
        """
        Interactive identification using the decision tree.

        Parameters:
        - interactive: If True, asks for input interactively. If False, simulates with random answers.
        """
        if self.decision_tree is None:
            print("Decision tree not built. Call build() first.")
            return

        print("=== Botanical Species Identification ===")
        print("Answer the following questions to identify the species.")
        print("You can skip questions if you're unsure or if the trait isn't visible.\n")

        node = self.decision_tree
        decisions = {}  # Dictionary to store trait decisions
        steps = []

        while node["type"] == "node":
            trait = node["trait"]
            display_name = node["display_name"]

            # Ask question
            print(f"Does the plant have {display_name}?")

            if interactive:
                while True:
                    answer = input("Enter y/n/s (y=yes, n=no, s=skip, q=quit): ").lower()
                    if answer in ['y', 'n', 's', 'q']:
                        break
                    print("Invalid input. Please enter 'y', 'n', 's', or 'q'.")

                if answer == 'q':
                    print("Identification terminated by user.")
                    return

                if answer == 's':
                    print("Skipping this question...\n")
                    # Record skipped question
                    steps.append({
                        "trait": trait,
                        "answer": "skipped"
                    })

                    # Get remaining species when skipping this trait
                    temp_decisions = decisions.copy()
                    remaining_species = self._follow_path(node, temp_decisions)

                    # If very few species left, simply show them without asking
                    if len(remaining_species) <= 10:
                        print(f"Skipping has narrowed down to {len(remaining_species)} species:")
                        for species in remaining_species:
                            print(f"- {species}")
                        print()

                    # Find next best trait to ask about
                    next_trait = self._find_best_trait(remaining_species)
                    if next_trait is None:
                        # No more distinguishing traits, show results
                        if not remaining_species:
                            print("No species match your answers.")
                        elif len(remaining_species) == 1:
                            species = remaining_species[0]
                            print(f"Identified species: {species}")
                            self._display_species_details(species)
                        else:
                            print(f"Multiple potential species identified ({len(remaining_species)}):")
                            for species in remaining_species:
                                print(f"- {species}")

                            # If there are only a few species left, show how they differ
                            if len(remaining_species) <= 5:
                                self._show_differentiating_traits(remaining_species)
                        return

                    # Find the node for this trait in the tree
                    # Simple approach - more complex search?
                    def find_node_with_trait(search_node, target_trait):
                        if search_node["type"] == "leaf":
                            return None
                        if search_node["trait"] == target_trait:
                            return search_node

                        yes_result = find_node_with_trait(search_node["yes"], target_trait)
                        if yes_result:
                            return yes_result

                        return find_node_with_trait(search_node["no"], target_trait)

                    # Try to find the next trait node
                    next_node = find_node_with_trait(self.decision_tree, next_trait)
                    if next_node:
                        node = next_node
                        continue
                    else:
                        # If we can't find exact node, simulate asking about species with this trait
                        temp_yes = [s for s in remaining_species if self.traits_data[s][next_trait] == 1]
                        temp_no = [s for s in remaining_species if self.traits_data[s][next_trait] == 0]

                        print(f"\nDoes the plant have {self._format_trait_name(next_trait)}?")

                        while True:
                            next_answer = input("Enter y/n/s (y=yes, n=no, s=skip, q=quit): ").lower()
                            if next_answer in ['y', 'n', 's', 'q']:
                                break
                            print("Invalid input. Please enter 'y', 'n', 's', or 'q'.")

                        if next_answer == 'q':
                            print("Identification terminated by user.")
                            return

                        if next_answer == 'y':
                            decisions[next_trait] = True
                            remaining_species = temp_yes
                        elif next_answer == 'n':
                            decisions[next_trait] = False
                            remaining_species = temp_no
                        # If skip again, continue with all remaining species

                        steps.append({
                            "trait": next_trait,
                            "answer": "yes" if next_answer == 'y' else "no" if next_answer == 'n' else "skipped"
                        })

                        # If very few species left or no distinguishing traits, show results
                        next_trait = self._find_best_trait(remaining_species)
                        if next_trait is None or len(remaining_species) <= 1:
                            if not remaining_species:
                                print("No species match your answers.")
                            elif len(remaining_species) == 1:
                                species = remaining_species[0]
                                print(f"Identified species: {species}")
                                self._display_species_details(species)
                            else:
                                print(f"Multiple potential species identified ({len(remaining_species)}):")
                                for species in remaining_species:
                                    print(f"- {species}")

                                # If there are only a few species left, show how they differ
                                if len(remaining_species) <= 5:
                                    self._show_differentiating_traits(remaining_species)
                            return

                        # Try to continue with simulated traits
                        continue

                # Store decision and follow the tree
                goes_yes = answer == 'y'
                decisions[trait] = goes_yes

            else:
                # For non-interactive mode, randomly choose a path
                goes_yes = np.random.choice([True, False])
                print(f"{'Yes' if goes_yes else 'No'} (auto-answer)")
                decisions[trait] = goes_yes

            # Record step
            steps.append({
                "trait": trait,
                "answer": "yes" if goes_yes else "no"
            })

            # Follow the appropriate branch
            node = node["yes"] if goes_yes else node["no"]
            print()

        # Reached a leaf node
        if not node["species"]:
            print("No species match your answers.")
        elif len(node["species"]) == 1:
            species = node["species"][0]
            print(f"Identified species: {species}")
            self._display_species_details(species)
        else:
            print(f"Multiple potential species identified ({len(node['species'])}):")
            for species in node["species"]:
                print(f"- {species}")

            # If there are only a few species left, show how they differ
            if len(node["species"]) <= 5:
                self._show_differentiating_traits(node["species"])

    def _display_species_details(self, species: str) -> None:
        """Display detailed information about a species."""
        traits_dict = self.traits_data[species]

        print("\nKey characteristics:")

        # Display binary traits that are present
        present_traits = [self._format_trait_name(t) for t, v in traits_dict.items()
                         if t in self.binary_traits and v == 1]
        if present_traits:
            print("\nPresent traits:")
            for trait in present_traits:
                print(f"- {trait}")

        # Display numeric traits
        numeric_traits = {self._format_trait_name(t): v for t, v in traits_dict.items()
                         if t not in self.binary_traits and not t.endswith('_months')
                         and v not in [None, 'NA']}
        if numeric_traits:
            print("\nMeasurements:")
            for trait, value in numeric_traits.items():
                print(f"- {trait}: {value}")

        # Display months
        fl_months = traits_dict.get('Fl_months', [])
        fr_months = traits_dict.get('Fr_months', [])
        if fl_months:
            print(f"\nFlowering months: {', '.join(fl_months)}")
        if fr_months:
            print(f"Fruiting months: {', '.join(fr_months)}")

    def _show_differentiating_traits(self, species_list: List[str]) -> None:
        """Show traits that differ among the given species."""
        print("\nDifferentiating traits:")

        # Create dataframe to compare species
        comparison_data = {}
        for species in species_list:
            comparison_data[species] = {
                self._format_trait_name(trait): self.traits_data[species][trait]
                for trait in self.binary_traits
                if self.traits_data[species][trait] not in ['NA', None]
            }

        df = pd.DataFrame(comparison_data)

        # Find traits that differ
        differing_traits = []
        for trait in self.binary_traits:
            formatted_trait = self._format_trait_name(trait)
            if formatted_trait not in df.index:
                continue

            values = set(df.loc[formatted_trait].dropna())
            if len(values) > 1:  # If trait has different values across species
                differing_traits.append(formatted_trait)

        if differing_traits:
            print("These species differ in these traits:")
            display(df.loc[differing_traits])
        else:
            print("These species have very similar traits in our dataset.")

    def export_key(self, filename: str) -> None:
        """
        Export the dichotomous key to a file.

        Parameters:
        - filename: Path to save the key
        """
        if self.decision_tree is None:
            print("Decision tree not built. Call build() first.")
            return

        with open(filename, 'w') as f:
            key_text = self._format_key_text(self.decision_tree)
            f.write(key_text)

        print(f"Dichotomous key exported to {filename}")

    def _format_key_text(self, node: Dict[str, Any], prefix: str = '', depth: int = 1) -> str:
        """
        Format the decision tree as a readable dichotomous key text.

        Parameters:
        - node: Current tree node
        - prefix: Current key prefix (for formatting)
        - depth: Current depth in the tree

        Returns:
        - Formatted text for the key
        """
        if node["type"] == "leaf":
            if not node["species"]:
                return f"{prefix} No species match these characteristics.\n"
            elif len(node["species"]) == 1:
                return f"{prefix} {node['species'][0]}\n"
            else:
                species_text = ", ".join(node["species"])
                return f"{prefix} Multiple species: {species_text}\n"

        # For internal nodes, create the couplet
        a_text = f"{prefix}{depth}a. {node['display_name']} present → "
        b_text = f"{prefix}{depth}b. {node['display_name']} absent → "

        # Recursive call for yes branch
        yes_branch = self._format_key_text(node["yes"], prefix + "  ", depth + 1)

        # If yes branch leads directly to a species, append it to the current line
        if node["yes"]["type"] == "leaf" and len(node["yes"]["species"]) == 1:
            a_text += node["yes"]["species"][0] + "\n"
        else:
            a_text += "Go to " + str(depth + 1) + "\n"
            a_text += yes_branch

        # Recursive call for no branch
        no_branch = self._format_key_text(node["no"], prefix + "  ", depth + 1)

        # If no branch leads directly to a species, append it to the current line
        if node["no"]["type"] == "leaf" and len(node["no"]["species"]) == 1:
            b_text += node["no"]["species"][0] + "\n"
        else:
            b_text += "Go to " + str(depth + 1) + "\n"
            b_text += no_branch

        return a_text + b_text

    def visualize_tree(self, max_depth: int = 5) -> None:
        """
        Visualize the decision tree in a more readable format.

        Parameters:
        - max_depth: Maximum depth to display (to prevent overwhelming output)
        """
        if self.decision_tree is None:
            print("Decision tree not built. Call build() first.")
            return

        print("Dichotomous Key Visualization:")
        self._print_tree(self.decision_tree, max_depth=max_depth)

    def _print_tree(self, node: Dict[str, Any], indent: str = '', is_last: bool = True,
                   max_depth: int = 5, current_depth: int = 0) -> None:
        """
        Recursively print the decision tree in a hierarchical format.

        Parameters:
        - node: Current tree node
        - indent: Current indentation string
        - is_last: Whether this is the last child of its parent
        - max_depth: Maximum depth to display
        - current_depth: Current depth in the tree
        """
        if current_depth > max_depth:
            print(f"{indent}└─ ... (truncated at depth {max_depth})")
            return

        # Print current node
        if node["type"] == "leaf":
            if not node["species"]:
                print(f"{indent}└─ No matching species")
            elif len(node["species"]) == 1:
                print(f"{indent}└─ Species: {node['species'][0]}")
            else:
                print(f"{indent}└─ Multiple species: {len(node['species'])}")
                for i, species in enumerate(node["species"]):
                    last_species = i == len(node["species"]) - 1
                    child_indent = indent + ("    " if last_species else "│   ")
                    print(f"{indent}{'└─ ' if last_species else '├─ '}{species}")
        else:
            # Internal node
            print(f"{indent}└─ Does it have {node['display_name']}?")

            # Print yes branch
            yes_indent = indent + "    "
            print(f"{indent}    ├─ Yes:")
            self._print_tree(node["yes"], yes_indent + "│   ", False, max_depth, current_depth + 1)

            # Print no branch
            no_indent = indent + "    "
            print(f"{indent}    └─ No:")
            self._print_tree(node["no"], no_indent + "    ", True, max_depth, current_depth + 1)

    def analyze_na_values(self) -> None:
        """Analyze and report on NA values in the dataset."""
        na_counts = {}

        # Count NA values for each trait
        for trait in self.binary_traits:
            na_count = sum(1 for species in self.all_species
                         if self.traits_data[species][trait] == "NA")
            na_percentage = (na_count / len(self.all_species)) * 100
            na_counts[trait] = (na_count, na_percentage)

        # Sort by percentage
        sorted_traits = sorted(na_counts.items(), key=lambda x: x[1][1], reverse=True)

        print("Analysis of NA values in traits:")
        print("================================")
        print(f"{'Trait':<30} {'NA Count':<10} {'Percentage':<10}")
        print("-" * 50)

        for trait, (count, percentage) in sorted_traits:
            formatted_trait = self._format_trait_name(trait)
            print(f"{formatted_trait:<30} {count:<10} {percentage:.2f}%")

# Example
def create_and_use_key(traits):
    """
    Create and use a dichotomous key from the traits data.

    Parameters:
    - traits: Dictionary mapping species to trait dictionaries
    """
    # Create the key
    key = DichotomousKey(traits)

    # Build the decision tree
    key.build()

    # Analyze NA values (optional)
    print("\nAnalyzing NA values in the dataset:")
    key.analyze_na_values()

    # Visualize the tree structure (first few levels)
    print("")
    key.visualize_tree(max_depth=3)

    # Start the interactive identification directly without asking
    print("\nStarting the interactive identification process:")
    key.identify(interactive=True)

    return key

# Usage in Colab
# key = create_and_use_key(traits)

# Standalone implementation for deployment
def main():
    """Main function to run when script is executed directly."""
    # Load traits from file
    traits_file = input("Enter the path to your traits JSON file: ")

    try:
        with open(traits_file, 'r') as f:
            traits = json.load(f)

        print(f"Loaded traits for {len(traits)} species.")
        create_and_use_key(traits)

    except FileNotFoundError:
        print(f"File not found: {traits_file}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {traits_file}")

if __name__ == "__main__":
    main()

# path (for now): /content/drive/MyDrive/Complete_Pedicularis_traits.json

# Visualize dichotomous key

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import re

def plot_dichotomous_tree(tree: dict, explicit_depth: int = 3, figsize=(16, 12), output_file="dichotomous_key.png"):
    G = nx.DiGraph()
    labels = {}
    node_types = {}  # To track node types (decision, ellipsis, species)

    def clean_species_name(name):
        """Extract just the species name without taxonomic authority and abbreviate genus"""
        # This pattern will match "Genus species" and ignore anything after
        match = re.match(r'(\w+)\s+(\w+)', name)
        if match:
            genus, species = match.group(1), match.group(2)
            # If it's Pedicularis, abbreviate it to P.
            if genus.lower() == "pedicularis":
                return f"P. {species}"
            return f"{genus} {species}"
        return name

    def get_terminal_species(node):
        """Get the terminal species from a leaf node"""
        if node["type"] == "leaf":
            if not node["species"]:
                return "No match"
            elif len(node["species"]) == 1:
                # Clean species name to remove taxonomic authority
                species_name = clean_species_name(node["species"][0])
                return species_name
            else:
                return f"Multiple species\n({len(node['species'])})"
        return None

    def add_nodes_edges(node, parent=None, label=None, depth=0, node_id=[0]):
        current_id = node_id[0]
        node_id[0] += 1

        # Handle leaf nodes
        if node["type"] == "leaf":
            name = get_terminal_species(node)
            # For leaf nodes, we don't add them as nodes
            # We'll handle them separately
            if parent is not None and depth <= explicit_depth:
                G.add_node(current_id)
                labels[current_id] = name
                node_types[current_id] = "species"
                G.add_edge(parent, current_id, label=label)
            return current_id

        # For decision nodes within explicit depth
        if depth < explicit_depth:
            name = node["display_name"]
            G.add_node(current_id)
            labels[current_id] = name
            node_types[current_id] = "decision"
            if parent is not None:
                G.add_edge(parent, current_id, label=label)

            # Recursively add child nodes
            add_nodes_edges(node["yes"], current_id, "Yes", depth + 1, node_id)
            add_nodes_edges(node["no"], current_id, "No", depth + 1, node_id)

        # For nodes at exactly the explicit depth
        elif depth == explicit_depth:
            # Create ellipsis node
            G.add_node(current_id)
            labels[current_id] = "⋮"
            node_types[current_id] = "ellipsis"
            if parent is not None:
                G.add_edge(parent, current_id, label=label)

            # Store species directly in the ellipsis node instead of creating a new node
            if label == "Yes" and node["type"] != "leaf":
                # Find the eventual terminal species by traversing the "Yes" branch
                current_node = node
                while current_node["type"] != "leaf":
                    current_node = current_node["yes"]
                yes_species = get_terminal_species(current_node)

                # Store the species name in a special attribute of the ellipsis node
                G.nodes[current_id]['species_name'] = yes_species
                G.nodes[current_id]['species_direction'] = "Yes"

            # For ellipsis with "No" branch
            elif label == "No" and node["type"] != "leaf":
                # Find the eventual terminal species by traversing the "No" branch
                current_node = node
                while current_node["type"] != "leaf":
                    current_node = current_node["no"]
                no_species = get_terminal_species(current_node)

                # Store the species name in a special attribute of the ellipsis node
                G.nodes[current_id]['species_name'] = no_species
                G.nodes[current_id]['species_direction'] = "No"

    add_nodes_edges(tree)

    # Use graphviz layout
    pos = nx.nx_pydot.graphviz_layout(G, prog='dot')

    # Create figure with high DPI for better quality PNG
    plt.figure(figsize=figsize, dpi=300)

    # Define node colors by type
    node_colors = []
    node_sizes = []
    ellipsis_nodes = []

    for node in G.nodes():
        if node_types[node] == "decision":
            node_colors.append("#8fd9a8")  # Light green for decision nodes
            node_sizes.append(2500)
        elif node_types[node] == "ellipsis":
            node_colors.append("#d3d3d3")  # Light gray for ellipsis nodes
            node_sizes.append(1800)
            ellipsis_nodes.append(node)
        else:  # species
            node_colors.append("#ffcc99")  # Light orange for species nodes
            node_sizes.append(2800)

    # Draw nodes with different colors
    nx.draw(G, pos, with_labels=False, arrows=True,
            node_color=node_colors, node_size=node_sizes,
            linewidths=1.0, edgecolors='black')

    # Draw edges
    nx.draw_networkx_edges(G, pos, width=1.5, arrowsize=20, arrowstyle='->', edge_color='#555555')

    # Draw node labels
    for node, label in labels.items():
        if node_types[node] == "species":
            plt.text(pos[node][0], pos[node][1], label,
                    fontsize=9, ha='center', va='center', style='italic',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=2.0))
        elif node_types[node] == "ellipsis":
            plt.text(pos[node][0], pos[node][1], label,
                    fontsize=14, ha='center', va='center',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=2.0))
        else:  # decision
            plt.text(pos[node][0], pos[node][1], label,
                    fontsize=10, ha='center', va='center',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=2.0))

    # Draw edge labels except for edges leading to species names
    edge_labels = nx.get_edge_attributes(G, 'label')
    filtered_edge_labels = {}

    for edge, label in edge_labels.items():
        source, target = edge
        # Only show labels for edges not connected to ellipsis nodes
        if target not in ellipsis_nodes:
            filtered_edge_labels[edge] = label

    # Draw edge labels with slightly larger font size
    nx.draw_networkx_edge_labels(G, pos, edge_labels=filtered_edge_labels,
                                font_color='#444444', font_size=10,
                                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=2))

    # Add species names below ellipsis nodes WITH shorter arrows and LARGER font
    for node in ellipsis_nodes:
        if 'species_name' in G.nodes[node]:
            x, y = pos[node]
            species_text = G.nodes[node]['species_name']
            direction = G.nodes[node]['species_direction']

            # Calculate shorter offset for the species names
            vertical_offset = 70

            # Draw shorter line connecting ellipsis to species name
            plt.plot([x, x], [y - 25, y - vertical_offset + 15],
                    color='#555555', linestyle='-', linewidth=1.5)

            # Add arrowhead
            plt.arrow(x, y - vertical_offset + 15, 0, -5,
                    head_width=7, head_length=7, fc='#555555', ec='#555555',
                    width=1.5, length_includes_head=True)

            # Draw species name right below the arrow with LARGER font size (9 -> 11)
            # But using normal font weight (not bold)
            plt.text(x, y - vertical_offset, species_text,
                    rotation=90,  # Vertical orientation
                    fontsize=11,  # Increased from 9 to 11
                    # No fontweight parameter = normal font weight
                    ha='center', va='top',  # Anchor at top center
                    style='italic',  # Italicize species names
                    bbox=dict(facecolor='white', alpha=0, edgecolor='none'))

            # Draw edge label (Yes/No) on the connecting line with adjusted position
            plt.text(x + 8, y - 40, direction,  # Adjusted y position to match shorter arrow
                    fontsize=9.5, ha='left', va='center', color='#444444',
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

    plt.axis('off')
    plt.tight_layout()

    # Save figure as PNG file
    plt.savefig(output_file, dpi=500, bbox_inches='tight', pad_inches=0.1)

    # Also display the figure
    plt.show()

    print(f"Visualization saved as {output_file}")

In [ ]:
key = DichotomousKey(traits)
key.build()

# Visualize the top 3 levels only
plot_dichotomous_tree(key.decision_tree, explicit_depth=3, figsize=(10, 7))

# Ordination and clustering of species based on binary traits; Visualization of trait similarity

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display, clear_output

def prepare_binary_traits_matrix(traits_data):
    all_traits = set()
    for species_traits in traits_data.values():
        all_traits.update(species_traits.keys())

    binary_traits = [
        trait for trait in all_traits
        if not (trait.endswith('_cm') or trait.endswith('_mm') or
                trait.endswith('_count') or trait == 'Chromosome_number_2n' or
                trait.endswith('_months'))
    ]

    species_list = list(traits_data.keys())
    trait_matrix = np.zeros((len(species_list), len(binary_traits)))

    for i, species in enumerate(species_list):
        for j, trait in enumerate(binary_traits):
            value = traits_data[species].get(trait, 'NA')
            if value == 'NA':
                trait_matrix[i, j] = np.nan
            else:
                trait_matrix[i, j] = value

    trait_df = pd.DataFrame(trait_matrix, index=species_list, columns=binary_traits)
    for column in trait_df.columns:
        most_common = trait_df[column].mode()[0]
        trait_df[column].fillna(most_common, inplace=True)

    return trait_df

def generate_umap_visualization(trait_df, n_neighbors=15, min_dist=0.1, random_state=42):
    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=2,
        random_state=random_state
    )
    umap_result = reducer.fit_transform(trait_df.values)
    umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])
    umap_df['Species'] = trait_df.index

    # Species epithet for abbreviated display
    umap_df['SpeciesDisplay'] = umap_df['Species'].apply(
        lambda x: f"P. {x.split(' ')[1]}" if x.startswith('Pedicularis') else x
    )

    return umap_df

def plot_umap_interactive(umap_df):
    # Dropdown widget (outside the plot)
    species_options = sorted(list(zip(umap_df['SpeciesDisplay'], umap_df['Species'])), key=lambda x: x[0])
    dropdown_options = [('Select a species...', '')] + species_options

    species_dropdown = widgets.Dropdown(
        options=dropdown_options,
        description='Species:',
        layout=widgets.Layout(width='50%')
    )

    # Output widget for the plot
    output = widgets.Output()

    # Dictionary to keep reference to the annotation
    plot_elements = {'annotation': None, 'highlight': None}

    # Update plot when dropdown selection changes
    def on_species_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_species = change['new']

            with output:
                clear_output(wait=True)

                # Fig and axis
                fig, ax = plt.subplots(figsize=(12, 10))

                # Plot area for species selection
                plt.subplots_adjust(right=0.75)

                # Add points
                scatter = ax.scatter(
                    umap_df['UMAP1'],
                    umap_df['UMAP2'],
                    c='blue',
                    alpha=0.7,
                    s=80
                )

                ax.set_title('UMAP Clustering of Species Based on Binary Traits', fontsize=16)
                ax.set_xlabel('UMAP Dimension 1', fontsize=12)
                ax.set_ylabel('UMAP Dimension 2', fontsize=12)
                ax.grid(True, linestyle='--', alpha=0.7)

                # Highlight selected species
                if selected_species:
                    selected_data = umap_df[umap_df['Species'] == selected_species]
                    if not selected_data.empty:
                        display_name = selected_data.iloc[0]['SpeciesDisplay']

                        # Highlight selected species with red circle
                        ax.scatter(
                            selected_data['UMAP1'],
                            selected_data['UMAP2'],
                            facecolors='none',
                            edgecolors='red',
                            s=200,
                            linewidths=2
                        )


                        ax.annotate(
                            display_name,
                            xy=(selected_data['UMAP1'].values[0], selected_data['UMAP2'].values[0]),
                            xytext=(10, 10),
                            textcoords="offset points",
                            fontsize=10,
                            style='italic',
                            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="red", alpha=0.9)
                        )

                # Hover annotations
                annot = ax.annotate(
                    "",
                    xy=(0,0),
                    xytext=(10,10),
                    textcoords="offset points",
                    style='italic',
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8)
                )
                annot.set_visible(False)

                def update_annot(ind):
                    pos = scatter.get_offsets()[ind["ind"][0]]
                    annot.xy = pos
                    text = umap_df.iloc[ind["ind"][0]]['SpeciesDisplay']
                    annot.set_text(text)
                    annot.set_visible(True)

                def hover(event):
                    vis = annot.get_visible()
                    if event.inaxes == ax:
                        cont, ind = scatter.contains(event)
                        if cont:
                            update_annot(ind)
                            fig.canvas.draw_idle()
                        else:
                            if vis:
                                annot.set_visible(False)
                                fig.canvas.draw_idle()

                fig.canvas.mpl_connect("motion_notify_event", hover)

                plt.tight_layout()
                plt.show()

    species_dropdown.observe(on_species_change)

    # Initialize plot
    on_species_change({'type': 'change', 'name': 'value', 'new': ''})

    # Dropdown and plot
    display(species_dropdown, output)

def main(json_file_path):
    with open(json_file_path, 'r') as f:
        traits_data = json.load(f)

    print(f"Loaded traits data for {len(traits_data)} species.")
    trait_df = prepare_binary_traits_matrix(traits_data)
    print(f"Prepared matrix with {trait_df.shape[1]} binary traits for {trait_df.shape[0]} species.")

    umap_df = generate_umap_visualization(trait_df)
    print("Generated UMAP visualization.")

    plot_umap_interactive(umap_df)
    print("Interactive visualization ready.")

    return trait_df, umap_df

# Example
# trait_df, umap_df = main('traits.json')

In [ ]:
main("/content/drive/MyDrive/Complete_Pedicularis_traits.json")